In [ ]:
%pip install opencv-python

In [1]:
import cv2
import os
import json

In [2]:
video_path = "./assets/squat_lowerbar_manh.mp4"
save_dir = "extracted_segments"
os.makedirs(save_dir, exist_ok=True)

In [3]:
exercise_name = "squat"
error_name = "lowerbar"
os.makedirs(os.path.join(save_dir, exercise_name, error_name), exist_ok=True)

In [ ]:
# Tạo tên folder sẽ lưu từng segment
dest_folder = os.path.join(save_dir, exercise_name, error_name, "video_1")
os.makedirs(dest_folder, exist_ok=True)


cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video file.")

# Thông tin video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"H264")

start_frame = None
end_frame = None
segment_count = 0
current_writer = None

# Danh sách metadata để lưu thông tin các segment
metadata = []
frames_buffer = []

display_width = 400  # Kích thước hiển thị video
display_height = int(height * (display_width / width))

# Đọc và hiển thị khung hình đầu tiên
ret, first_frame = cap.read()
if ret:
    resized_frame = cv2.resize(first_frame, (display_width, display_height))
    cv2.imshow("Video", resized_frame)
    print("Press any to start playing the video...")
    cv2.waitKey(0)
else:
    print("Error: Cannot read the first frame.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Thay đổi kích thước hiển thị video
    resized_frame = cv2.resize(frame, (display_width, display_height))

    # Hiển thị video
    cv2.imshow("Video", resized_frame)

    key = cv2.waitKey(1) & 0xFF

    # Nhấn 's' để bắt đầu collect khung hình
    if key == ord("s"):
        if not frames_buffer:
            print(f"Frame start: {current_frame}")

    # Lưu khung hình vào bộ đệm nếu đang collect
    if len(frames_buffer) > 0 or key == ord("s"):
        frames_buffer.append(frame)

    # Nhấn 'e' để kết thúc collect và lưu đoạn video
    if key == ord("e") and frames_buffer:
        print(f"Frame end: {current_frame}. Please wait while saving the segment...")
        segment_path = os.path.join(dest_folder, f"segment_{segment_count}.mp4")
        out = cv2.VideoWriter(segment_path, fourcc, fps, (width, height))

        for buffered_frame in frames_buffer:
            out.write(buffered_frame)

        out.release()

        # Lưu thông tin metadata
        start_frame = current_frame - len(frames_buffer)
        end_frame = current_frame
        metadata.append({
            "segment_path": segment_path,
            "video_src_path": video_path,
            "start_frame": start_frame,
            "end_frame": end_frame,
            "start_time_seconds": round(start_frame / fps, 2),
            "end_time_seconds": round(end_frame / fps, 2),
        })

        # Reset bộ đệm
        frames_buffer = []
        segment_count += 1

        print("-----> Paused. Press any key to continue...")
        key = cv2.waitKey(0)


    # Nhấn 'q' để thoát
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Ghi metadata ra file JSON
metadata_path = os.path.join(dest_folder, "metadata.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Metadata saved to {metadata_path}")